In [26]:
import pandas as pd

import scipy
from scipy import sparse

import numpy as np
from sklearn.model_selection import TimeSeriesSplit

import pickle


### Read data

In [27]:
weather_alarms_tfidf = pd.read_csv('data/df_weather+alarms_merged.csv')
tfidf_vectors = pd.read_csv('data/0_isw_data_collection/3_isw_vectorised_with_date.csv')

In [28]:
weather_alarms_tfidf.shape

(203420, 70)

In [29]:
weather_alarms_tfidf.head()

,Unnamed: 0,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,...,event_day_date,event_start_hour_datetimeEpoch,event_end_hour_datetimeEpoch,event_hour_level_event_time,event_hour_level_event_datetimeEpoch,isw_report_date,isw_date_tomorrow_datetime,isw_keywords,isw_main_html_v6,isw_report_text_lemm
0,0,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"Черкаси, Україна",2022-02-24,1645653600,4.9,-0.1,1.9,-0.6,83.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
tfidf_vectors.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,7392,7393,7394,7395,7396,7397,7398,7399,7400,date
0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-25
1,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-26
2,2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-27
3,3,0.0,0.0,0.0,0.0,0.0,0.023238,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-28
4,4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.033858,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-03-01


In [31]:
weather_alarms_tfidf.columns

Index(['Unnamed: 0', 'city_resolvedAddress', 'day_datetime',
       'day_datetimeEpoch', 'day_tempmax', 'day_tempmin', 'day_temp',
       'day_dew', 'day_humidity', 'day_precip', 'day_precipcover',
       'day_solarradiation', 'day_solarenergy', 'day_uvindex', 'day_sunrise',
       'day_sunset', 'day_moonphase', 'hour_datetime', 'hour_datetimeEpoch',
       'hour_temp', 'hour_humidity', 'hour_dew', 'hour_precip',
       'hour_precipprob', 'hour_snow', 'hour_snowdepth', 'hour_preciptype',
       'hour_windgust', 'hour_windspeed', 'hour_winddir', 'hour_pressure',
       'hour_visibility', 'hour_cloudcover', 'hour_solarradiation',
       'hour_solarenergy', 'hour_uvindex', 'hour_severerisk',
       'hour_conditions', 'city', 'region', 'center_city_ua', 'center_city_en',
       'region_alt', 'region_id', 'event_Unnamed: 0', 'event_region_title',
       'event_region_city', 'event_all_region', 'event_start', 'event_end',
       'event_clean_end', 'event_intersection_alarm_id', 'event_start_

In [32]:
weather_alarms_tfidf.shape

(203420, 70)

### TF_IDF + CV with isw_report_text_lemm preprocessed

In [40]:
tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))
tfidf_vector = scipy.sparse.load_npz('data/matrix/tfidf_vector_train.npz')

#Uncommit if you want to create new one
#word_count_vector = cv.transform(weather_alarms_tfidf['isw_report_text_lemm'].values.astype('U'))
#tfidf_vector = tfidf.transform(word_count_vector)

C:\Users\dimai\AppData\Local\Temp\ipykernel_7984\3905827888.py:1: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))


In [41]:
#Save model and matrix
#with open("models/tfidf_vector_calculated.pkl", "wb") as handle:
#    pickle.dump(tfidf_vector, handle)
#scipy.sparse.save_npz('data/matrix/tfidf_vector_train.npz', tfidf_vector)

In [42]:
tfidf_vector

<203420x7401 sparse matrix of type '<class 'numpy.float64'>'
	with 123370706 stored elements in Compressed Sparse Row format>

In [10]:
weather_alarms_tfidf['isAlarm'] = weather_alarms_tfidf['event_start_hour'].notnull().astype(int)
Y = weather_alarms_tfidf['isAlarm']
weather_alarms_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203420 entries, 0 to 203419
Data columns (total 71 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   Unnamed: 0                                      203420 non-null  int64  
 1   city_resolvedAddress                            203420 non-null  object 
 2   day_datetime                                    203420 non-null  object 
 3   day_datetimeEpoch                               203420 non-null  int64  
 4   day_tempmax                                     203420 non-null  float64
 5   day_tempmin                                     203420 non-null  float64
 6   day_temp                                        203420 non-null  float64
 7   day_dew                                         203420 non-null  float64
 8   day_humidity                                    203420 non-null  float64
 9   day_precip                

In [26]:
weather_alarms_tfidf_features = weather_alarms_tfidf.drop(columns=['event_all_region', 'event_start',
       'event_end', 'event_clean_end', 'event_intersection_alarm_id',
       'event_start_time', 'event_end_time', 'event_day',
       'event_feature_number_of_region', 'event_within_24_hours',
       'event_feature_number_of_alarms_within_24_hours', 'event_start_hour',
       'event_end_hour', 'event_day_date', 'event_start_hour_datetimeEpoch',
       'event_end_hour_datetimeEpoch', 'event_hour_level_event_time',
       'event_hour_level_event_datetimeEpoch','region_alt','hour_conditions','center_city_en','hour_datetime','day_sunset','day_sunrise','isAlarm','hour_precip','hour_solarradiation','hour_solarenergy','hour_uvindex', 'event_region_title', 'event_region_city', 'isw_report_date', 'isw_date_tomorrow_datetime', 'isw_keywords', 'isw_main_html_v6', 'isw_report_text_lemm', 'city', 'region', 'center_city_ua', 'hour_preciptype', 'city_resolvedAddress', 'day_datetime', 'event_Unnamed: 0'],axis=1)

In [27]:
weather_alarms_tfidf_features_clear = weather_alarms_tfidf_features

In [28]:
weather_alarms_tfidf_features.isnull().sum()

Unnamed: 0            0
day_datetimeEpoch     0
day_tempmax           0
day_tempmin           0
day_temp              0
day_dew               0
day_humidity          0
day_precip            0
day_precipcover       0
day_solarradiation    0
day_solarenergy       0
day_uvindex           0
day_moonphase         0
hour_datetimeEpoch    0
hour_temp             0
hour_humidity         0
hour_dew              0
hour_precipprob       0
hour_snow             0
hour_snowdepth        0
hour_windgust         0
hour_windspeed        0
hour_winddir          0
hour_pressure         0
hour_visibility       0
hour_cloudcover       0
hour_severerisk       0
region_id             0
dtype: int64

In [29]:
weather_alarms_tfidf_csr = scipy.sparse.csr_matrix(weather_alarms_tfidf_features_clear)
weather_alarms_tfidf_features = scipy.sparse.hstack((weather_alarms_tfidf_csr, tfidf_vector), format="csr")

In [30]:
weather_alarms_tfidf_features

<203420x7429 sparse matrix of type '<class 'numpy.float64'>'
	with 128262032 stored elements in Compressed Sparse Row format>

### Devide data into train and test parts 

In [31]:
from sklearn.model_selection import TimeSeriesSplit

# Create a time series dataset (e.g., a NumPy array or a pandas DataFrame)
# time_series_data = pd.DataFrame(weather_alarms_tfidf_features)

# Split the dataset into input features (X) and target variable (y)
X = weather_alarms_tfidf_features  # Assuming the target variable is in the last column
y = Y  # Assuming the target variable is in the last column

# Initialize the TimeSeriesSplit object
n_splits = 5  # Number of splits
tscv = TimeSeriesSplit(n_splits=n_splits)

# Iterate over the TimeSeriesSplit object to generate train and test indices
for train_index, test_index in tscv.split(X):
    # train_index and test_index are arrays of indices for train and test sets, respectively

    # Access the train and test data for X and y without using iloc
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Perform your model training and evaluation using X_train, X_test, y_train, and y_test

In [32]:
X_train.shape

(169517, 7429)

In [33]:
y_train.shape

(169517,)

# Train models

In [2]:
#Imports
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import SGDClassifier # For SGD
from sklearn.model_selection import train_test_split # For data splitting
from sklearn.metrics import accuracy_score, classification_report # For model evaluation
from sklearn.ensemble import RandomForestClassifier # For Random Forest
from sklearn.model_selection import train_test_split # For data splitting
from sklearn.metrics import accuracy_score, classification_report # For model evaluation
from sklearn.naive_bayes import GaussianNB # For Gaussian Naive Bayes
from sklearn.model_selection import train_test_split # For data splitting
from sklearn.metrics import accuracy_score, classification_report # For model evaluation
from sklearn.neighbors import KNeighborsClassifier # For K-nearest neighbors
from sklearn.model_selection import train_test_split # For data splitting
from sklearn.metrics import accuracy_score, classification_report # For model evaluation

COMMAND_ID = "0"
VERSION = "1.0"

### Model 1 LogisticRegression

In [3]:
# Create an instance of the LogisticRegression class
logreg = LogisticRegression()

# Train the logistic regression model
logreg.fit(X_train, y_train)

# Predict the target variable on the test data
y_pred = logreg.predict(X_test)

# Evaluate the performance of the model
accuracy = accuracy_score(X_test, y_test)
classification_report = classification_report(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report)

# Save model
MODEL_NAME = "logreg"
with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
    pickle.dump(logreg, f)

NameError: name 'X_train' is not defined

### Model 2 SVC

In [ ]:
# Import the required libraries
from sklearn.svm import SVC # For SVM
from sklearn.model_selection import train_test_split # For data splitting
from sklearn.metrics import accuracy_score, classification_report # For model evaluation

# Create an SVM classifier with default hyperparameters
svm = SVC()

# Fit the SVM classifier to the training data
svm.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = svm.predict(X_test)

# Evaluate the performance of the SVM classifier
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report)

# Save model
MODEL_NAME = "svm"
with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
    pickle.dump(svm, f)

### Model 3 SGDClassifier

In [ ]:
# Create an SGD classifier with default hyperparameters
sgd = SGDClassifier()

# Fit the SGD classifier to the training data
sgd.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = sgd.predict(X_test)

# Evaluate the performance of the SGD classifier
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report)

# Save model
MODEL_NAME = "sgd"
with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
    pickle.dump(sgd, f)

### Model 4 RandomForestClassifier

In [ ]:
# Create a Random Forest classifier with default hyperparameters
rf = RandomForestClassifier()

# Fit the Random Forest classifier to the training data
rf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf.predict(X_test)

# Evaluate the performance of the Random Forest classifier
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report)

# Save model
MODEL_NAME = "rf"
with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
    pickle.dump(rf, f)

### Model 5 GaussianNB

In [ ]:
# Create a Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Fit the Gaussian Naive Bayes classifier to the training data
gnb.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = gnb.predict(X_test)

# Evaluate the performance of the Gaussian Naive Bayes classifier
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report)

# Save model
MODEL_NAME = "gnb"
with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
    pickle.dump(gnb, f)

### Model 6 KNeighborsClassifier

In [ ]:
# Create a K-nearest neighbors classifier with k=3 (default value)
knn = KNeighborsClassifier()

# Fit the K-nearest neighbors classifier to the training data
knn.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = knn.predict(X_test)

# Evaluate the performance of the K-nearest neighbors classifier
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report)

# Save model
MODEL_NAME = "knn"
with open(f'models/training_models/{COMMAND_ID}_{MODEL_NAME}_{VERSION}.pkl', 'wb') as f:
    pickle.dump(knn, f)